# Pruning with our automatic strutured Pruning framework
Welcome to an end-to-end example for magnitude-based weight pruning

**Summary**

In this tutorial, you will:

* Train a tf.keras model for Cifa10 from scratch.
* Fine tune the model by applying the pruning Framework and see the accuracy.





In [1]:
!git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
!echo $CWD
!cp -rf /content/Automatic-Structured-Pruning/* /content/

#The Framework work with Tensorflow 2.1-2.3.1
import Pruning

Cloning into 'Automatic-Structured-Pruning'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.

2.3.0


# Train a model for CIFA10 without pruning
Download and prepare the CIFAR10 dataset
The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

Create the convolutional base
The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument input_shape to our first layer.

To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
#import matplotlib.pyplot as plt
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.summary()



170500096/170498071 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_______________________________________________________

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.
As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.


# Compile and train the model

In [3]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

Epoch 1/10
1563/1563 [==============================] - 66s 42ms/step - loss: 1.5565 - accuracy: 0.4301 - val_loss: 1.3369 - val_accuracy: 0.5140
Epoch 2/10
1563/1563 [==============================] - 65s 42ms/step - loss: 1.1773 - accuracy: 0.5777 - val_loss: 1.1735 - val_accuracy: 0.5831
Epoch 3/10
1563/1563 [==============================] - 67s 43ms/step - loss: 1.0143 - accuracy: 0.6406 - val_loss: 1.0585 - val_accuracy: 0.6293
Epoch 4/10
1563/1563 [==============================] - 65s 42ms/step - loss: 0.9085 - accuracy: 0.6803 - val_loss: 0.9588 - val_accuracy: 0.6663
Epoch 5/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.8311 - accuracy: 0.7078 - val_loss: 0.8845 - val_accuracy: 0.6955
Epoch 6/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.7721 - accuracy: 0.7289 - val_loss: 0.8692 - val_accuracy: 0.6985
Epoch 7/10
1563/1563 [==============================] - 65s 41ms/step - loss: 0.7204 - accuracy: 0.7467 - val_loss: 0.8741 -

In [4]:
model.save('my_model.h5') 

# Fine-tune pre-trained model with pruning
You will apply pruning to the whole model and see this in the model summary.

In this example, you prune the model with 10% Dense pruning and 10% Filter pruning.



In [5]:
Denseprunerate=10
Convprunerate=10

prunemodel=Pruning.prune_model('./my_model.h5',Denseprunerate, Convprunerate)

Finish with pruning
[ 0  0  0  0  0  0 58  0  0]
No dense or conv
No dense or conv
No dense or conv
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
______

We see how we get less Parameter in the pruned Model

# Compile and re-train the model

In [6]:
prunemodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = prunemodel.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

Epoch 1/10
1563/1563 [==============================] - 67s 43ms/step - loss: 0.9869 - accuracy: 0.6501 - val_loss: 0.8713 - val_accuracy: 0.7002
Epoch 2/10
1563/1563 [==============================] - 67s 43ms/step - loss: 0.7278 - accuracy: 0.7444 - val_loss: 0.8263 - val_accuracy: 0.7162
Epoch 3/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.6433 - accuracy: 0.7747 - val_loss: 0.8543 - val_accuracy: 0.7140
Epoch 4/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.5882 - accuracy: 0.7931 - val_loss: 0.8245 - val_accuracy: 0.7232
Epoch 5/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.5445 - accuracy: 0.8095 - val_loss: 0.8631 - val_accuracy: 0.7239
Epoch 6/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.5035 - accuracy: 0.8237 - val_loss: 0.8689 - val_accuracy: 0.7271
Epoch 7/10
1563/1563 [==============================] - 67s 43ms/step - loss: 0.4675 - accuracy: 0.8374 - val_loss: 0.9120 -

Compare both Models

In [7]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

313/313 - 4s - loss: 0.8756 - accuracy: 0.7103
313/313 - 4s - loss: 1.0083 - accuracy: 0.7051
